In [ ]:
!pip install -q --upgrade git+https://github.com/huggingface/transformers.git

!pip install -q datasets lightning



In [ ]:
!pip install -q peft accelerate bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
    
notebook_login()

In [8]:
import torch
import numpy as np
from PIL import Image
import requests
from transformers import AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor, AutoProcessor

PROMPT = "extract JSON."
img_path = "1.jpg"
input_image = Image.open(img_path)
MAX_LENGTH = 512


In [6]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "nielsr/paligemma-cord-demo"
processor_id = "google/paligemma-3b-pt-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(processor_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [7]:
inputs = processor(text=PROMPT, images=input_image, return_tensors="pt")
for k,v in inputs.items():
    print(k,v.shape)

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


input_ids torch.Size([1, 261])
attention_mask torch.Size([1, 261])
pixel_values torch.Size([1, 3, 224, 224])


In [9]:
generated_ids = model.generate(**inputs, max_new_tokens=MAX_LENGTH)


# Turn each predicted Token ID back into a string using decode method
#We chop off the prompt, which consists of image tokens and our text prompt
image_token_index = model.config.image_token_index
num_image_tokens = len(generated_ids[generated_ids==image_token_index])
num_text_tokens = len(processor.tokenizer.encode(PROMPT))
num_prompt_tokens = num_image_tokens = num_text_tokens + 2
generated_text = processor.batch_decode(generated_ids[:, num_prompt_tokens:], skip_special_tokens=True, clean_up_tokenization_spaces=False)
generated_text

['extract JSON.\n<s_total_price>440.00</s_total_price><s_cashprice>440.00</s_cashprice><s_creditcardprice>0.00</s_creditcardprice><s_changeprice>0.00</s_changeprice><s_menu><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</']

In [15]:
generated_text[0]

'extract JSON.\n<s_total_price>440.00</s_total_price><s_cashprice>440.00</s_cashprice><s_creditcardprice>0.00</s_creditcardprice><s_changeprice>0.00</s_changeprice><s_menu><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</s_nm><s_cnt>1</s_cnt><sep/><s_unitprice>2.99</s_unitprice><s_price>2.99</s_price><s_nm>E-CARROTS, CHIMICHURRI, 10 OZ</'

In [10]:
import re

# let's turn that into JSON
def token2json(tokens, is_inner_value=False, added_vocab=None):
        """
        Convert a (generated) token sequence into an ordered JSON format.
        """
        if added_vocab is None:
            added_vocab = processor.tokenizer.get_added_vocab()

        output = {}

        while tokens:
            start_token = re.search(r"<s_(.*?)>", tokens, re.IGNORECASE)
            if start_token is None:
                break
            key = start_token.group(1)
            key_escaped = re.escape(key)

            end_token = re.search(rf"</s_{key_escaped}>", tokens, re.IGNORECASE)
            start_token = start_token.group()
            if end_token is None:
                tokens = tokens.replace(start_token, "")
            else:
                end_token = end_token.group()
                start_token_escaped = re.escape(start_token)
                end_token_escaped = re.escape(end_token)
                content = re.search(
                    f"{start_token_escaped}(.*?){end_token_escaped}", tokens, re.IGNORECASE | re.DOTALL
                )
                if content is not None:
                    content = content.group(1).strip()
                    if r"<s_" in content and r"</s_" in content:  # non-leaf node
                        value = token2json(content, is_inner_value=True, added_vocab=added_vocab)
                        if value:
                            if len(value) == 1:
                                value = value[0]
                            output[key] = value
                    else:  # leaf nodes
                        output[key] = []
                        for leaf in content.split(r"<sep/>"):
                            leaf = leaf.strip()
                            if leaf in added_vocab and leaf[0] == "<" and leaf[-2:] == "/>":
                                leaf = leaf[1:-2]  # for categorical special tokens
                            output[key].append(leaf)
                        if len(output[key]) == 1:
                            output[key] = output[key][0]

                tokens = tokens[tokens.find(end_token) + len(end_token) :].strip()
                if tokens[:6] == r"<sep/>":  # non-leaf nodes
                    return [output] + token2json(tokens[6:], is_inner_value=True, added_vocab=added_vocab)

        if len(output):
            return [output] if is_inner_value else output
        else:
            return [] if is_inner_value else {"text_sequence": tokens}



In [16]:
generated_json = token2json(generated_text[0])
print(generated_json)

[{'total_price': '440.00', 'cashprice': '440.00', 'creditcardprice': '0.00', 'changeprice': '0.00', 'unitprice': '2.99', 'price': '2.99', 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ', 'cnt': '1'}, {'unitprice': '2.99', 'price': '2.99', 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ', 'cnt': '1'}, {'unitprice': '2.99', 'price': '2.99', 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ', 'cnt': '1'}, {'unitprice': '2.99', 'price': '2.99', 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ', 'cnt': '1'}, {'unitprice': '2.99', 'price': '2.99', 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ', 'cnt': '1'}, {'unitprice': '2.99', 'price': '2.99', 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ', 'cnt': '1'}, {'unitprice': '2.99', 'price': '2.99'}]


In [30]:
generated_json[1]

{'unitprice': '2.99',
 'price': '2.99',
 'nm': 'E-CARROTS, CHIMICHURRI, 10 OZ',
 'cnt': '1'}